# Coursera Capstone

#### This will be used to build project on Data Science 

In [188]:
import pandas as pd
import numpy as np

In [189]:
print("Hello! Capstone Project Course!")

Hello! Capstone Project Course!


In [190]:
from bs4 import BeautifulSoup
import requests 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = requests.get(url).text
type(request)

str

In [191]:
page = BeautifulSoup(request, 'lxml')

In [192]:
table = page.find('table', class_='wikitable sortable')

#### lets grab all the columns of table in list


In [193]:
columns = []
for column in table.find_all('th'):
    columns.append(column.text.replace('\n',''))
    
columns

['Postcode', 'Borough', 'Neighbourhood']

#### lets grab all rows in a list

In [274]:

all = []
for tr in table.find_all('tr'):    
    row=[]
    for col in tr.find_all('td'):
        row.append(col.text.replace('\n',''))    
    all.append(row)

In [275]:
df = pd.DataFrame(all, columns=columns)
df.drop(0, inplace=True)  #dropping 0th index row. for some reason it was adding a None Value row

#### Removing all rows having Borough = Not assigned

In [276]:
df = df[df.Borough != 'Not assigned']

#### lets check how many value in Neighbourhood have "Not assigned"

In [277]:
df[df.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
9,M7A,Queen's Park,Not assigned


In [278]:
df['Neighbourhood'] = np.where(df['Neighbourhood']=='Not assigned', df['Borough'], df['Neighbourhood'])

#### lets check how many value in Neighbourhood have "Not assigned"

In [279]:
df[df.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood


#### Adding Neighbourhood for same Borough

In [280]:
df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda col: ', '.join(col)).reset_index(name='Neighbourhood').head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [281]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda col: ', '.join(col)).reset_index(name='Neighbourhood')

In [282]:
df.shape

(103, 3)

In [267]:
## Will try GeCoder some other day
#import geocoder # import geocoder

## initialize your variable to None
#lat_lng_coords = None

## loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [268]:
#Saving sometime and using the CSV provided to create the data frame

In [252]:
lldf = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

In [288]:
df.sort_values('Postcode', inplace=True)


In [290]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [291]:
lldf.sort_values('Postal Code', inplace=True)

In [292]:
lldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [293]:
df['Latitude'] = lldf['Latitude']
df['Longitude'] = lldf['Longitude']

In [296]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [297]:
### Lets cluster the neighbourhood of borough having words toronto

In [298]:
df_tor = df[df['Borough'].str.contains("Toronto")]

In [304]:
df_tor.shape

(38, 5)

In [322]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes
from geopy.geocoders import Nominatim
import folium

In [318]:
address = 'Toronto, Canada'
geo_locator = Nominatim(user_agent="tr_exp")
location = geo_locator.geocode(address, timeout=60)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [323]:
# create map of Toronto using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    


In [324]:
map